In [1]:
import json
import operator

In [2]:
pool_depth = 100

In [3]:
def clean_sub(raw_sub, index_of_ad, index_of_score, expected_length):
    seen = []
    uniqs = []
    # explicitly check ordering
    i = 100
    # Pull out entries not of expected length
    sub = []
    for x in raw_sub:
        if type(x) == list:
            if len(x) == expected_length:
                sub.append(x)
    sub.sort(key=operator.itemgetter(index_of_score), reverse=True)
    
    for entry in sub:
        if len(entry) == expected_length:
            if entry[index_of_score] > i:
                print "ORDERING PROBLEM"
                return None
            i = entry[index_of_score]
            #else:
            #    print type(entry)
            
    # de-dupe
    for entry in sub:
        if len(entry) == expected_length:
            if entry[index_of_ad] not in seen:
                uniqs.append(entry)
                seen.append(entry[index_of_ad])
                
    # explicitly sort by score after de-duping
    uniqs.sort(key=operator.itemgetter(index_of_score), reverse=True)
    
    # explicitly check ordering
    i = 100
    for entry in uniqs:
        if entry[index_of_score] > i:
            print "ORDERING PROBLEM"
            return None
        i = entry[index_of_score]
        
    return uniqs

In [4]:
def get_subs(team, entry, id_pos, score_pos, ans_length):
    submissions = clean_sub(entry[configs[team]['afield']], id_pos, score_pos, ans_length)
    # Reconfirming no dupes
    if len(submissions) != len(list(set(tuple(i) for i in submissions))):
        print "PROBLEM WITH DUPLICATED DOC IDS"
    # Now that submissions are sorted, only take top N
    if len(submissions) > pool_depth:
        submissions = submissions[:pool_depth]
        #print "POOL DEPTH REACHED"
    
    return submissions

## Note which questions are to be evaluated

In [5]:
questions_file =  '../annotation_prep/dd_clustering/FIRST_ROUND_chosen_questions.json'

with open(questions_file, 'r') as f:
    questions = eval(f.read())

question_ids = {}
for qtype in questions['NYU'].keys():
    question_ids[qtype] = questions['NYU'][qtype].keys()

## Obtain Submissions From Each Team

In [18]:
configs = {}
configs['Georgetown'] = {}
configs['Georgetown']['qfield'] = 'id'
configs['Georgetown']['afield'] = 'answer'
configs['Georgetown']['Cluster Identification'] = {}
configs['Georgetown']['Cluster Identification']['id_pos'] = 0
configs['Georgetown']['Cluster Identification']['score_pos'] = 1
configs['Georgetown']['Cluster Identification']['exp_length'] = 2
configs['Georgetown']['Cluster Facet'] = {}
configs['Georgetown']['Cluster Facet']['id_pos'] = 1
configs['Georgetown']['Cluster Facet']['score_pos'] = 2
configs['Georgetown']['Cluster Facet']['exp_length'] = 3
configs['Georgetown']['Cluster Aggregate'] = {}
configs['Georgetown']['Cluster Aggregate']['id_pos'] = 1
configs['Georgetown']['Cluster Aggregate']['score_pos'] = 2
configs['Georgetown']['Cluster Aggregate']['exp_length'] = 3

configs['ISI'] = {}
configs['ISI']['qfield'] = 'question_id'
configs['ISI']['afield'] = 'answer'
configs['ISI']['Cluster Identification'] = {}
configs['ISI']['Cluster Identification']['id_pos'] = 1
configs['ISI']['Cluster Identification']['score_pos'] = 2
configs['ISI']['Cluster Identification']['exp_length'] = 3
configs['ISI']['Cluster Facet'] = {}
configs['ISI']['Cluster Facet']['id_pos'] = 1
configs['ISI']['Cluster Facet']['score_pos'] = 2
configs['ISI']['Cluster Facet']['exp_length'] = 3
configs['ISI']['Cluster Aggregate'] = {}
configs['ISI']['Cluster Aggregate']['id_pos'] = 1
configs['ISI']['Cluster Aggregate']['score_pos'] = 2
configs['ISI']['Cluster Aggregate']['exp_length'] = 3

configs['Uncharted'] = {}
configs['Uncharted']['qfield'] = 'question_id'
configs['Uncharted']['afield'] = 'answers'
configs['Uncharted']['Cluster Identification'] = {}
configs['Uncharted']['Cluster Identification']['id_pos'] = 0
configs['Uncharted']['Cluster Identification']['score_pos'] = 1
configs['Uncharted']['Cluster Identification']['exp_length'] = 2
configs['Uncharted']['Cluster Facet'] = {}
configs['Uncharted']['Cluster Facet']['id_pos'] = 1
configs['Uncharted']['Cluster Facet']['score_pos'] = 2
configs['Uncharted']['Cluster Facet']['exp_length'] = 3
configs['Uncharted']['Cluster Aggregate'] = {}
configs['Uncharted']['Cluster Aggregate']['id_pos'] = 1
configs['Uncharted']['Cluster Aggregate']['score_pos'] = 2
configs['Uncharted']['Cluster Aggregate']['exp_length'] = 3

In [19]:
submission_file_paths = {}
submission_file_paths['Georgetown'] = {}
submission_file_paths['Georgetown']['NYU'] = {}
submission_file_paths['Georgetown']['NYU']['Cluster Identification'] = 'Georgetown/DomainDiscovery/NYU_CI.json'
submission_file_paths['Georgetown']['NYU']['Cluster Facet'] = 'Georgetown/DomainDiscovery/NYU_CF.json'
submission_file_paths['Georgetown']['NYU']['Cluster Aggregate'] = 'Georgetown/DomainDiscovery/NYU_aggregate.json'
submission_file_paths['Georgetown']['HG'] = {}
submission_file_paths['Georgetown']['HG']['Cluster Identification'] = ('Georgetown/DomainDiscovery/'
                                                                       'Georgetown_Submission/HG/'
                                                                       'HG_Cluster_Identification.json')
submission_file_paths['Georgetown']['HG']['Cluster Facet'] = ('Georgetown/DomainDiscovery/'
                                                              'Georgetown_Submission/HG/'
                                                              'HG_Cluster_Facet.json')
submission_file_paths['Georgetown']['HG']['Cluster Aggregate'] = ('Georgetown/DomainDiscovery/'
                                                                  'Georgetown_Submission/HG/'
                                                                  'HG_Aggregate.json')
submission_file_paths['Georgetown']['JPL'] = {}
submission_file_paths['Georgetown']['JPL']['Cluster Identification'] = ('Georgetown/DomainDiscovery/'
                                                                       'Georgetown_Submission/JPL/'
                                                                       'JPL_Cluster_Identification.json')
submission_file_paths['Georgetown']['JPL']['Cluster Facet'] = ('Georgetown/DomainDiscovery/'
                                                              'Georgetown_Submission/JPL/'
                                                              'JPL_Cluster_Facet.json')
submission_file_paths['Georgetown']['JPL']['Cluster Aggregate'] = ('Georgetown/DomainDiscovery/'
                                                                  'Georgetown_Submission/JPL/'
                                                                  'JPL_Aggregate.json')

submission_file_paths['ISI'] = {}
submission_file_paths['ISI']['NYU'] = {}
submission_file_paths['ISI']['NYU']['Cluster Identification'] = ('ISI/DomainDiscovery/isi-nyu-answers-dig-extractions/'
                                                                 'properly_formatted_submissions/'
                                                                 'formatted_post_cluster_identification'
                                                                 '-parsed_fixed_all_answers.json')
submission_file_paths['ISI']['NYU']['Cluster Facet'] = ('ISI/DomainDiscovery/isi-nyu-answers-dig-extractions/'
                                                        'properly_formatted_submissions/'
                                                        'formatted_post_cluster_facet'
                                                        '_parsed_fixed_all_answers.json')
submission_file_paths['ISI']['NYU']['Cluster Aggregate'] = ('ISI/DomainDiscovery/isi-nyu-answers-dig-extractions/'
                                                            'properly_formatted_submissions/'
                                                            'formatted_post_aggregate'
                                                            '_parsed_fixed_all_answers.json')
submission_file_paths['ISI']['HG'] = {}
submission_file_paths['ISI']['HG']['Cluster Identification'] = ('ISI/DomainDiscovery/hg_all_asnwers/'
                                                                 'properly_formatted_submissions/'
                                                                 'formatted_post_cluster_identification'
                                                                 '-parsed_fixed_all_answers.json')
submission_file_paths['ISI']['HG']['Cluster Facet'] = ('ISI/DomainDiscovery/hg_all_asnwers/'
                                                        'properly_formatted_submissions/'
                                                        'formatted_post_cluster_facet'
                                                        '_parsed_fixed_all_answers.json')
submission_file_paths['ISI']['HG']['Cluster Aggregate'] = ('ISI/DomainDiscovery/hg_all_asnwers/'
                                                            'properly_formatted_submissions/'
                                                            'formatted_post_aggregate'
                                                            '_parsed_fixed_all_answers.json')
submission_file_paths['ISI']['JPL'] = {}
submission_file_paths['ISI']['JPL']['Cluster Identification'] = ('ISI/DomainDiscovery/jpl_answers_isi/'
                                                                 'properly_formatted_submissions/'
                                                                 'formatted_cluster-identification-queries-parsed_all_answers.json')
submission_file_paths['ISI']['JPL']['Cluster Facet'] = ('ISI/DomainDiscovery/jpl_answers_isi/'
                                                        'properly_formatted_submissions/'
                                                        'formatted_cluster-facet-queries-parsed_all_answers.json')
submission_file_paths['ISI']['JPL']['Cluster Aggregate'] = ('ISI/DomainDiscovery/jpl_answers_isi/'
                                                            'properly_formatted_submissions/'
                                                            'formatted_aggregate-queries-parsed_all_answers.json')

In [20]:
answers = {}

for team in submission_file_paths.keys():
#for team in ['ISI']:
    print ' '
    print team.upper()
    print ' '
    answers[team] = {}
    for dataset in submission_file_paths[team].keys():
        print dataset
        answers[team][dataset] = {}
        for qtype in submission_file_paths[team][dataset].keys():
            print qtype
            answers[team][dataset][qtype] = {}
            # Load team's submissions for that qtype and dataset
            file_path = submission_file_paths[team][dataset][qtype]
            with open(file_path, 'r') as f:
                data = eval(f.read())
                
            for entry in data:
                # If the question id is to be evaluated
                if team == 'ISI':
                    qid = entry[configs[team]['qfield']] .split('-')[0]
                else:
                    qid = entry[configs[team]['qfield']]
                if qid in question_ids[qtype]:
                    # Some questions have no answers
                    if len(entry[configs[team]['afield']]) > 0:
                        answers[team][dataset][qtype][qid] = {}
                        submissions = get_subs(team, entry, configs[team][qtype]['id_pos'],
                                              configs[team][qtype]['score_pos'], configs[team][qtype]['exp_length'])
                    else:
                        submissions = []
                    
                    answers[team][dataset][qtype][qid]['submissions'] = submissions
                    if qtype == 'Cluster Aggregate':
                        if type(entry[configs[team]['afield']][0]) != list:
                            agg_answ = entry[configs[team]['afield']][0]
                        # It could be a list of multiple submissions
                        elif len(entry[configs[team]['afield']][0]) < configs[team][qtype]['exp_length']:
                            agg_answ = entry[configs[team]['afield']][0]
                        else:
                            agg_answ = "NA"
                            # Confirmed "NO AGGS" for Georgetown, 
                            print "NO AGG FOUND FOR QUESTION {0}".format(entry[configs[team]['qfield']])
                        answers[team][dataset][qtype][qid]['agg_answ'] = agg_answ

 
GEORGETOWN
 
JPL
Cluster Aggregate
Cluster Facet
Cluster Identification
HG
Cluster Aggregate
Cluster Facet
Cluster Identification
NYU
Cluster Aggregate
NO AGG FOUND FOR QUESTION 100
NO AGG FOUND FOR QUESTION 37
NO AGG FOUND FOR QUESTION 4
NO AGG FOUND FOR QUESTION 44
NO AGG FOUND FOR QUESTION 5
NO AGG FOUND FOR QUESTION 68
NO AGG FOUND FOR QUESTION 69
NO AGG FOUND FOR QUESTION 76
NO AGG FOUND FOR QUESTION 79
NO AGG FOUND FOR QUESTION 82
NO AGG FOUND FOR QUESTION 83
NO AGG FOUND FOR QUESTION 84
Cluster Facet
Cluster Identification
 
ISI
 
JPL
Cluster Aggregate
Cluster Facet
Cluster Identification
HG
Cluster Aggregate
Cluster Facet
Cluster Identification
NYU
Cluster Aggregate
Cluster Facet
Cluster Identification


In [21]:
answers['ISI']['HG']['Cluster Identification']

{'17': {'submissions': [['',
    '9A570B18849E607AC9DBC32F3D964861854531A4542895A5D329C1683917210B',
    1.0],
   ['',
    '511ABCD3DDC3CB3835DE6DF666CB4415C9F2BCA77201DDF0FE4A102EFDF7FB2E',
    0.5],
   ['',
    '175C7ABD85981AF042857BAC75A11868FF11C6BEAE35A391AC4030E9A623C9F9',
    0.3333333333333333],
   ['',
    'F09404B008191CAA2FC29181CF3133CD0A4DE1B02E3841296442280BA099BC50',
    0.25],
   ['',
    'C0B4AE54C208C1072E85B8C91732720ED826DE9EC9F86476E8644029D11BC742',
    0.2],
   ['',
    '861A823870BB695B7A8BEE32EAF3C6E4FE9F686DB711CA504FF3860D2A074341',
    0.16666666666666666],
   ['',
    '4634D142CEE96FD37D08FAB452CA4A346B6412E273C9CDA6DCA430D5D6137A95',
    0.14285714285714285],
   ['',
    'DD637AA84E4BA4A7D02108B846C4072A92AC790EF82EB8F8459930E4D5674805',
    0.125],
   ['',
    '400E62180839654EB35A010F5410E6CFF0BBA15B4327F1E2246FDF9FB2887BDE',
    0.1111111111111111],
   ['',
    '0160DFD1B913627A0E6F87BB31129DC5138B346524BFF6F4C9E98179AA9E36B9',
    0.1],
   ['',
    '9

In [22]:
output_file = 'collected_cluster_answers_REWRITE.json'
# Will re-name manually to avoid overwriting
with open(output_file, 'w') as f:
    json.dump(answers, f, indent=2)

## Combine Relevant Ads

In [ ]:
first_round_file = 'FIRST_ROUND_seeds2cluster_ads.json'
second_round_file = 'FIRST_ROUND_seeds2cluster_ads.json'

In [ ]:
with open(first_round_file, 'r') as f:
    first_round = eval(f.read())

with open(second_round_file, 'r') as f:
    second_round = eval(f.read())
    
new_seeds2cluster_ad = {}

for seed in first_round.keys():
    temp1 = first_round[seed]
    temp2 = second_round[seed]
    # List comprehension can cause backwards updates
    new_temp = [x for x in temp1]
    new_temp.extend(temp2)
    new_temp = list(set(new_temp))
    
    # There shouldn't be any dupes
    if len(new_temp) != len(temp1) + len(temp2):
        print 'TROUBLE'
    
    new_seeds2cluster_ad[seed] = new_temp

# Attribute Relevant Ads to a Dataset

## Obtain Submissions

In [ ]:
pwd